In [1]:
!pip install simpy

In [2]:
import simpy
import random
import time
from statistics import mean

# Paràmetres
ARRIVAL_TIME = 8.64
QUEUE        = 6
CAM          = 63
TOTAL_TIME   = 86400              # 24 horas
ASSEMBLE     = 40

# Funcions
def fn_perf():                    # fn_perf() = fn$perf
  #smaller number equals higher priority
  return random.choices([0, 1], weights=[0.2, 0.8])[0]
def fn_xpdis():
  return random.expovariate(1.0)
def fn_destinacio():
  return random.choice([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

# Recursos globals
class FactoryResources:
  def __init__(self, env):        # simpy.Resource = cola amb servidors
    self.queue = simpy.PriorityResource(env, capacity=QUEUE)
    self.cam = simpy.Resource(env, capacity=CAM)
    self.pto1 = simpy.Resource(env, capacity=1)
    self.pto2 = simpy.Resource(env, capacity=1)

    # Variables estadístiques
    self.PTO1_usage_time = 0
    self.PTO2_usage_time = 0
    self.queue_usage_time = 0
    self.pesa_usage_time = 0
    self.cam_usage_time = 0

    self.PTO1_entries = 0
    self.PTO2_entries = 0
    self.queue_entries = 0
    self.pesa_entries = 0
    self.cam_entries = 0

    self.last_queue_update = 0
    self.queue_length_time = 0
    self.queue_current_length = 0
    self.env = env

# Metriques
class Stats:
    def __init__(self):
        self.queue_times_pref = []
        self.queue_times_econ = []
        self.usage_times_pref = []
        self.usage_times_econ = []
        self.system_times_pref = []
        self.system_times_econ = []


stats = Stats()

def assemble_manager(env, store, machine, name):
  while True:
    # Esperar hasta que haya suficientes piezas para ensamblar
    while len(store.items) < ASSEMBLE:
      yield env.timeout(1)

    # Sacar un lote de 40 piezas
    for _ in range(ASSEMBLE):
      yield store.get()

          # Usar el recurso de montaje
    with machine.request() as req:
      yield req
      t_start = env.now
      setattr(resources, f"{name}_usage_time", getattr(resources, f"{name}_usage_time") + 1)
      yield env.timeout(600)  # 600s = 10min
      t_end = env.now
      setattr(resources, f"{name}_usage_time", getattr(resources, f"{name}_usage_time") + (t_end - t_start))
      setattr(resources, f"{name}_entries", getattr(resources, f"{name}_entries") + 1)

# Decideix a que moll va el paquet
def repart_moll(env, resources, pto1_store, pto2_store):
    if random.random() < 0.5:
        yield pto1_store.put(env.now)
    else:
        yield pto2_store.put(env.now)

# Colas
def normal_process(env, resources, priority, pto1_store, pto2_store):
    t_start_p = env.now
    with resources.queue.request(priority=priority) as req:
        yield req
        t_start = env.now
        wait_time = t_start - t_start_p  # only time waiting in queue

        # service time
        yield env.timeout(45 * fn_xpdis())
        t_end = env.now
        usage_time = t_end - t_start
        system_time = t_end - t_start_p

        resources.queue_usage_time += usage_time
        resources.queue_entries += 1

        if priority == 0:  # preferred
            stats.queue_times_pref.append(wait_time)
            stats.usage_times_pref.append(usage_time)
            stats.system_times_pref.append(system_time)
        else:  # economic
            stats.queue_times_econ.append(wait_time)
            stats.usage_times_econ.append(usage_time)
            stats.system_times_econ.append(system_time)

    yield env.process(repart_moll(env, resources, pto1_store, pto2_store))


def pesa_process(env, resources, priority, pto1_store, pto2_store):
    t_start_p = env.now
    with resources.queue.request(priority=priority) as req:
        yield req
        t_start = env.now
        wait_time = t_start - t_start_p # only time waiting in queue

        # service time
        yield env.timeout(45 * fn_xpdis() * 1.5)
        t_end = env.now
        usage_time = t_end - t_start
        system_time = t_end - t_start_p

        resources.pesa_usage_time += usage_time
        resources.pesa_entries += 1

        if priority == 0:  # preferred
            stats.queue_times_pref.append(wait_time)
            stats.usage_times_pref.append(usage_time)
            stats.system_times_pref.append(system_time)
        else:  # economic
            stats.queue_times_econ.append(wait_time)
            stats.usage_times_econ.append(usage_time)
            stats.system_times_econ.append(system_time)

    yield env.process(repart_moll(env, resources, pto1_store, pto2_store))


# Proceso principal de la entidad
def part_process(env, resources, pto1_store, pto2_store):
  priority = fn_perf()
  destinacio = fn_destinacio()

  if random.random() < 0.01:
    yield env.process(pesa_process(env, resources, priority, pto1_store, pto2_store))
  else:
    yield env.process(normal_process(env, resources, priority, pto1_store, pto2_store))

  with resources.cam.request() as req:
    yield req
    t_start = env.now
    resources.cam_entries += 1
    yield env.timeout(21600)        # 21600s = 6h
    t_end = env.now
    resources.cam_usage_time += (t_end - t_start)

# Generador d'entitats
def generator(env, resources, pto1_store, pto2_store):
  for _ in range(10000):
    yield env.timeout(ARRIVAL_TIME)
    env.process(part_process(env, resources, pto1_store, pto2_store))

In [3]:
# Medició de temps real
start = time.time()

# Crear entorn y recursos
env = simpy.Environment()
resources = FactoryResources(env)
pto1_store = simpy.Store(env)
pto2_store = simpy.Store(env)

# Iniciar procesos
env.process(generator(env, resources, pto1_store, pto2_store))
env.process(assemble_manager(env, pto1_store, resources.pto1, "PTO1"))
env.process(assemble_manager(env, pto2_store, resources.pto2, "PTO2"))

# Ejecutar simulació
env.run(until=TOTAL_TIME)

# Calcular
pto1_utilization = (resources.PTO1_usage_time * 100) / TOTAL_TIME
pto2_utilization = (resources.PTO2_usage_time * 100) / TOTAL_TIME
queue_utilization = (resources.queue_usage_time/QUEUE * 100) / TOTAL_TIME
pesa_utilization = (resources.pesa_usage_time/QUEUE * 100) / TOTAL_TIME
cam_utilization = (resources.cam_usage_time/CAM * 100) / TOTAL_TIME

pto1_time_trans = resources.PTO1_usage_time / resources.PTO1_entries
pto2_time_trans = resources.PTO2_usage_time / resources.PTO2_entries
queue_time_trans = resources.queue_usage_time / resources.queue_entries
pesa_time_trans = resources.pesa_usage_time / resources.pesa_entries
cam_time_trans = resources.cam_usage_time / resources.cam_entries

queue_ocupacio_actual = len(resources.queue.users)
cam_ocupacio_actual = len(resources.cam.users)

pto1_contents_mitjans = (resources.PTO1_entries / TOTAL_TIME) * pto1_time_trans
pto2_contents_mitjans = (resources.PTO2_entries / TOTAL_TIME) * pto2_time_trans
queue_contents_mitjans = (resources.queue_entries / TOTAL_TIME) * queue_time_trans
pesa_contents_mitjans = (resources.pesa_entries / TOTAL_TIME) * pesa_time_trans
cam_contents_mitjans = (resources.cam_entries / TOTAL_TIME) * cam_time_trans

avg_queue_time_pref = mean(stats.queue_times_pref) if stats.queue_times_pref else 0
avg_queue_time_econ = mean(stats.queue_times_econ) if stats.queue_times_econ else 0
avg_usage_times_pref = mean(stats.usage_times_pref) if stats.usage_times_pref else 0
avg_usage_times_econ = mean(stats.usage_times_econ) if stats.usage_times_econ else 0
avg_system_times_pref = mean(stats.system_times_pref) if stats.system_times_pref else 0
avg_system_times_econ = mean(stats.system_times_econ) if stats.system_times_econ else 0

# Resultat
end = time.time()
print(f"Simulació completada en {end - start:.3f} segons reals.\n")

print("- Cua normal:")
print("Capacitat de la cua normal: 6")
print(f"Contents mitjans de la cua normal: {queue_contents_mitjans:.2f}")
print(f"Us mitja de la cua normal: {queue_utilization:.2f}%")
print(f"Contents actuals de la cua normal: {queue_ocupacio_actual}")
print(f"Contents totals de la cua normal: {resources.queue_entries}")
print(f"Temps mitja per unitat de la cua normal: {queue_time_trans:.2f}\n")

print("- Cua pesada:")
print("Capacitat de la cua pesada: 6")
print(f"Contents mitjans de la cua pesada: {pesa_contents_mitjans:.2f}")
print(f"Us mitja de la cua pesada: {pesa_utilization:.2f}%")
print(f"Contents totals de la cua pesada: {resources.pesa_entries}")
print(f"Temps mitja per unitat de la cua pesada: {pesa_time_trans:.2f}\n")

print("- Camions:")
print("Capacitat dels camions: 63")
print(f"Contents mitjans dels camions: {cam_contents_mitjans:.2f}")
print(f"Us mitja del camió: {cam_utilization:.2f}%")
print(f"Contents actuals del camió: {cam_ocupacio_actual}")
print(f"Contents totals del camió: {resources.cam_entries}")
print(f"Temps mitja per unitat dels camions: {cam_time_trans:.2f}\n")

print("- Moll 1:")
print("Capacitat del moll 1: 1")
print(f"Contents mitjans del moll 1: {pto1_contents_mitjans:.2f}")
print(f"Us mitja de moll 1: {pto1_utilization:.2f}%")
print(f"Contents totals del moll 1: {resources.PTO1_entries}")
print(f"Temps mitja per unitat del moll 1: {pto1_time_trans:.2f}\n")

print("- Moll 2:")
print("Capacitat del moll 2: 1")
print(f"Contents mitjans del moll 2: {pto2_contents_mitjans:.2f}")
print(f"Us mitja de moll 2: {pto2_utilization:.2f}%")
print(f"Contents totals del moll 2: {resources.PTO2_entries}")
print(f"Temps mitja per unitat del moll 2: {pto2_time_trans:.2f}\n")

print("- Paquets:")
print(f"Cola paquets preferents: {avg_queue_time_pref:.2f}s")
print(f"Cola paquets econòmics: {avg_queue_time_econ:.2f}s")
print(f"Demora paquets preferents: {avg_usage_times_pref:.2f}s")
print(f"Demora paquets econòmics: {avg_usage_times_econ:.2f}s")
print(f"Total paquets preferents: {avg_system_times_pref:.2f}s")
print(f"Total paquets econòmics: {avg_system_times_econ:.2f}s")


Simulació completada en 0.839 segons reals.

- Cua normal:
Capacitat de la cua normal: 6
Contents mitjans de la cua normal: 5.16
Us mitja de la cua normal: 86.06%
Contents actuals de la cua normal: 6
Contents totals de la cua normal: 9883
Temps mitja per unitat de la cua normal: 45.14

- Cua pesada:
Capacitat de la cua pesada: 6
Contents mitjans de la cua pesada: 0.09
Us mitja de la cua pesada: 1.45%
Contents totals de la cua pesada: 111
Temps mitja per unitat de la cua pesada: 67.82

- Camions:
Capacitat dels camions: 63
Contents mitjans dels camions: 47.25
Us mitja del camió: 75.00%
Contents actuals del camió: 63
Contents totals del camió: 252
Temps mitja per unitat dels camions: 16200.00

- Moll 1:
Capacitat del moll 1: 1
Contents mitjans del moll 1: 0.86
Us mitja de moll 1: 86.25%
Contents totals del moll 1: 124
Temps mitja per unitat del moll 1: 601.00

- Moll 2:
Capacitat del moll 2: 1
Contents mitjans del moll 2: 0.86
Us mitja de moll 2: 86.25%
Contents totals del moll 2: 124
Te